In [1]:
#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.nnet import conv
from theano.tensor.nnet import softmax
from theano.tensor import shared_randomstreams
from theano.tensor.signal import downsample
from theano.tensor.nnet import sigmoid
from theano.tensor import tanh

In [2]:
# 添加ReLu激活函数
def linear(z): return z
def ReLU(z): return T.maximum(0.0, z)

#使用GPU进行计算
GPU = True
if GPU:
    print ("Trying to run under a GPU.  If this is not desired, then modify "+\
        "network3.py\nto set the GPU flag to False.")
    try: theano.config.device = 'gpu'
    except: pass # it's already set
    theano.config.floatX = 'float32'
else:
    print ("Running with a CPU.  If this is not desired, then the modify "+\
        "network3.py to set\nthe GPU flag to True.")

#加载mnist数据集
#划分为训练集 交叉验证集 测试集
def load_data_shared(filename="G:/data/mnist.pkl.gz"):
    f = gzip.open(filename, 'rb')
    training_data,validation_data,test_data=pickle.load(f,encoding='bytes')
    f.close()
    def shared(data):
        """Place the data into shared variables.  This allows Theano to copy
        the data to the GPU, if one is available.
        """
        shared_x = theano.shared(
            np.asarray(data[0], dtype=theano.config.floatX), borrow=True)
        shared_y = theano.shared(
            np.asarray(data[1], dtype=theano.config.floatX), borrow=True)
        return shared_x, T.cast(shared_y, "int32")
    return [shared(training_data), shared(validation_data), shared(test_data)]

#主网络
class Network(object):

    def __init__(self, layers, mini_batch_size):

        self.layers = layers
        self.mini_batch_size = mini_batch_size
        self.params = [param for layer in self.layers for param in layer.params]
        self.x = T.matrix("x")
        self.y = T.ivector("y")
        init_layer = self.layers[0]
        init_layer.set_inpt(self.x, self.x, self.mini_batch_size)
        for j in range(1, len(self.layers)):
            prev_layer, layer  = self.layers[j-1], self.layers[j]
            layer.set_inpt(
                prev_layer.output, prev_layer.output_dropout, self.mini_batch_size)
        #最后一层的输出为输出
        self.output = self.layers[-1].output
        self.output_dropout = self.layers[-1].output_dropout

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            validation_data, test_data, lmbda=0.0):
        #使用批量随机梯度下降
        training_x, training_y = training_data
        validation_x, validation_y = validation_data
        test_x, test_y = test_data

        # 训练 交叉验证 测试的批次数
        num_training_batches = size(training_data)//mini_batch_size
        num_validation_batches = size(validation_data)//mini_batch_size
        num_test_batches = size(test_data)//mini_batch_size

        # 定义代价函数（最小二乘法） 计算梯度 以及更新参数
        l2_norm_squared = sum([(layer.w**2).sum() for layer in self.layers])
        cost = self.layers[-1].cost(self)+\
               0.5*lmbda*l2_norm_squared/num_training_batches
        grads = T.grad(cost, self.params)
        updates = [(param, param-eta*grad)
                   for param, grad in zip(self.params, grads)]

        #定义函数来训练一个mini-batch，然后计算交叉验证集和测试集的准确度
        i = T.lscalar() # mini-batch的索引
        train_mb = theano.function(
            [i], cost, updates=updates,
            givens={
                self.x:
                training_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                training_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        validate_mb_accuracy = theano.function(
            [i], self.layers[-1].accuracy(self.y),
            givens={
                self.x:
                validation_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                validation_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        test_mb_accuracy = theano.function(
            [i], self.layers[-1].accuracy(self.y),
            givens={
                self.x:
                test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                test_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        self.test_mb_predictions = theano.function(
            [i], self.layers[-1].y_out,
            givens={
                self.x:
                test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })
        # 训练
        best_validation_accuracy = 0.0
        for epoch in range(epochs):
            for minibatch_index in range(num_training_batches):
                iteration = num_training_batches*epoch+minibatch_index
                #每训练一千个mini-batch输出一次
                if iteration % 1000 == 0:
                    print("Training mini-batch number {0}".format(iteration))
                cost_ij = train_mb(minibatch_index)
                #每训练完一个epoch输出一次交叉验证集的准确度
                if (iteration+1) % num_training_batches == 0:
                    validation_accuracy = np.mean(
                        [validate_mb_accuracy(j) for j in range(num_validation_batches)])
                    print("Epoch {0}: validation accuracy {1:.2%}".format(
                        epoch, validation_accuracy))
                    #判断是否是当前最高的准确度
                    if validation_accuracy >= best_validation_accuracy:
                        print("This is the best validation accuracy to date.")
                        best_validation_accuracy = validation_accuracy
                        best_iteration = iteration
                        #如果是则输出相应的测试集的准确度
                        if test_data:
                            test_accuracy = np.mean(
                                [test_mb_accuracy(j) for j in range(num_test_batches)])
                            print('The corresponding test accuracy is {0:.2%}'.format(
                                test_accuracy))
        #训练完成输出最好的交叉验证集准确度 以及相应的测试集准确度
        print("Finished training network.")
        print("Best validation accuracy of {0:.2%} obtained at iteration {1}".format(
            best_validation_accuracy, best_iteration))
        print("Corresponding test accuracy of {0:.2%}".format(test_accuracy))

#其他种类的层
class ConvPoolLayer(object):
    #卷积和池化层写在一起的层

    def __init__(self, filter_shape, image_shape, poolsize=(2, 2),
                 activation_fn=sigmoid):

        self.filter_shape = filter_shape
        self.image_shape = image_shape
        self.poolsize = poolsize
        self.activation_fn=activation_fn
        # 初始化权值和偏置
        n_out = (filter_shape[0]*np.prod(filter_shape[2:])/np.prod(poolsize))
        self.w = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=np.sqrt(1.0/n_out), size=filter_shape),
                dtype=theano.config.floatX),
            borrow=True)
        self.b = theano.shared(
            np.asarray(
                np.random.normal(loc=0, scale=1.0, size=(filter_shape[0],)),
                dtype=theano.config.floatX),
            borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape(self.image_shape)
        conv_out = conv.conv2d(
            input=self.inpt, filters=self.w, filter_shape=self.filter_shape,
            image_shape=self.image_shape)
        pooled_out = downsample.max_pool_2d(
            input=conv_out, ds=self.poolsize, ignore_border=True)
        self.output = self.activation_fn(
            pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))
        self.output_dropout = self.output #在卷积层是没有dropout的

class FullyConnectedLayer(object):
    #全连接层
    def __init__(self, n_in, n_out, activation_fn=sigmoid, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.activation_fn = activation_fn
        self.p_dropout = p_dropout
        # 初始化权值和偏置
        self.w = theano.shared(
            np.asarray(
                np.random.normal(
                    loc=0.0, scale=np.sqrt(1.0/n_out), size=(n_in, n_out)),
                dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.asarray(np.random.normal(loc=0.0, scale=1.0, size=(n_out,)),
                       dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = self.activation_fn(
            (1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(
            inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = self.activation_fn(
            T.dot(self.inpt_dropout, self.w) + self.b)

    def accuracy(self, y):
        "Return the accuracy for the mini-batch."
        return T.mean(T.eq(y, self.y_out))

class SoftmaxLayer(object):
    #输出层
    def __init__(self, n_in, n_out, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.p_dropout = p_dropout
        # 初始化权值和偏置
        self.w = theano.shared(
            np.zeros((n_in, n_out), dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.zeros((n_out,), dtype=theano.config.floatX),
            name='b', borrow=True)
        self.params = [self.w, self.b]

    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = softmax((1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(
            inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = softmax(T.dot(self.inpt_dropout, self.w) + self.b)

    def cost(self, net):
        "Return the log-likelihood cost."
        return -T.mean(T.log(self.output_dropout)[T.arange(net.y.shape[0]), net.y])

    def accuracy(self, y):
        "Return the accuracy for the mini-batch."
        return T.mean(T.eq(y, self.y_out))


def size(data):
    "Return the size of the dataset `data`."
    return data[0].get_value(borrow=True).shape[0]
#添加dropout
def dropout_layer(layer, p_dropout):
    srng = shared_randomstreams.RandomStreams(
        np.random.RandomState(0).randint(999999))
    mask = srng.binomial(n=1, p=1-p_dropout, size=layer.shape)
    return layer*T.cast(mask, theano.config.floatX)

Trying to run under a GPU.  If this is not desired, then modify network3.py
to set the GPU flag to False.


In [4]:
training_data, validation_data, test_data =load_data_shared()
mini_batch_size = 10

只有全连接层

In [25]:
net = Network([
        FullyConnectedLayer(n_in=784, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.84%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.86%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 94.70%
This is the best validation accuracy to date.
The corresponding test accuracy is 93.97%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 95.72%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.18%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 151000
Training mini-batch number 152000
Training mini-batch number 153000
Training mini-batch number 154000
Epoch 30: validation accuracy 97.58%
Training mini-batch number 155000
Training mini-batch number 156000
Training mini-batch number 157000
Training mini-batch number 158000
Training mini-batch number 159000
Epoch 31: validation accuracy 97.59%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.83%
Training mini-batch number 160000
Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 97.59%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.84%
Training mini-batch number 165000
Training mini-batch number 166000
Training mini-batch number 167000
Training mini-batch number 168000
Training mini-batch number 169000
Epoch 33: validation accuracy 97.59%
This is the best validation ac

添加一层卷积层和一层池化层

In [7]:
net1 = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=20*12*12, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net1.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data) 

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 93.58%
This is the best validation accuracy to date.
The corresponding test accuracy is 92.82%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.07%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.57%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.08%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.66%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 142000
Training mini-batch number 143000
Training mini-batch number 144000
Epoch 28: validation accuracy 98.71%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.68%
Training mini-batch number 145000
Training mini-batch number 146000
Training mini-batch number 147000
Training mini-batch number 148000
Training mini-batch number 149000
Epoch 29: validation accuracy 98.75%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.67%
Training mini-batch number 150000
Training mini-batch number 151000
Training mini-batch number 152000
Training mini-batch number 153000
Training mini-batch number 154000
Epoch 30: validation accuracy 98.75%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.70%
Training mini-batch number 155000
Training mini-batch number 156000
Training mini-batch number 157000
Training mini-batch number 158000
Training mini-batch number 159000
Epoch 31: val

添加两层卷积层和池化层
可以看到其实准确度上升不大

In [8]:
net2 = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2)),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2)),
        FullyConnectedLayer(n_in=40*4*4, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net2.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)        

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.68%
This is the best validation accuracy to date.
The corresponding test accuracy is 92.49%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.89%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.51%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.50%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.29%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Training mini-batch number 149000
Epoch 29: validation accuracy 98.84%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.89%
Training mini-batch number 150000
Training mini-batch number 151000
Training mini-batch number 152000
Training mini-batch number 153000
Training mini-batch number 154000
Epoch 30: validation accuracy 98.84%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.88%
Training mini-batch number 155000
Training mini-batch number 156000
Training mini-batch number 157000
Training mini-batch number 158000
Training mini-batch number 159000
Epoch 31: validation accuracy 98.84%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.88%
Training mini-batch number 160000
Training mini-batch number 161000
Training mini-batch number 162000
Training mini-batch number 163000
Training mini-batch number 164000
Epoch 32: validation accuracy 98.84%
This is the best validation accuracy to date

Training mini-batch number 298000
Training mini-batch number 299000
Epoch 59: validation accuracy 98.90%
Finished training network.
Best validation accuracy of 98.91% obtained at iteration 279999
Corresponding test accuracy of 98.97%


使用ReLu函数作为激活函数，准确率突破到了99%

In [9]:
net3 = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net3.SGD(training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.48%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.33%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 98.15%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.15%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 98.36%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.05%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

Epoch 33: validation accuracy 99.01%
Training mini-batch number 170000
Training mini-batch number 171000
Training mini-batch number 172000
Training mini-batch number 173000
Training mini-batch number 174000
Epoch 34: validation accuracy 99.01%
Training mini-batch number 175000
Training mini-batch number 176000
Training mini-batch number 177000
Training mini-batch number 178000
Training mini-batch number 179000
Epoch 35: validation accuracy 99.00%
Training mini-batch number 180000
Training mini-batch number 181000
Training mini-batch number 182000
Training mini-batch number 183000
Training mini-batch number 184000
Epoch 36: validation accuracy 99.00%
Training mini-batch number 185000
Training mini-batch number 186000
Training mini-batch number 187000
Training mini-batch number 188000
Training mini-batch number 189000
Epoch 37: validation accuracy 99.00%
Training mini-batch number 190000
Training mini-batch number 191000
Training mini-batch number 192000
Training mini-batch number 193000

使用数据增强技术进行对数据扩充到原来的5倍

In [3]:
expanded_training_data,validation_data, test_data= load_data_shared(filename="G:/data/mnist_expanded.pkl.gz")
mini_batch_size = 10

In [6]:
net4 = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net4.SGD(expanded_training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-batch number 19000
Training mini-batch number 20000
Training mini-batch number 21000
Training mini-batch number 22000
Training mini-batch number 23000
Training mini-batch number 24000
Epoch 0: validation accuracy 98.88%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.14%
Training mini-batch number 25000
Training mini-batch number 2600

Training mini-batch number 223000
Training mini-batch number 224000
Epoch 8: validation accuracy 99.20%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.29%
Training mini-batch number 225000
Training mini-batch number 226000
Training mini-batch number 227000
Training mini-batch number 228000
Training mini-batch number 229000
Training mini-batch number 230000
Training mini-batch number 231000
Training mini-batch number 232000
Training mini-batch number 233000
Training mini-batch number 234000
Training mini-batch number 235000
Training mini-batch number 236000
Training mini-batch number 237000
Training mini-batch number 238000
Training mini-batch number 239000
Training mini-batch number 240000
Training mini-batch number 241000
Training mini-batch number 242000
Training mini-batch number 243000
Training mini-batch number 244000
Training mini-batch number 245000
Training mini-batch number 246000
Training mini-batch number 247000
Training mini-batch number

Epoch 17: validation accuracy 99.16%
Training mini-batch number 450000
Training mini-batch number 451000
Training mini-batch number 452000
Training mini-batch number 453000
Training mini-batch number 454000
Training mini-batch number 455000
Training mini-batch number 456000
Training mini-batch number 457000
Training mini-batch number 458000
Training mini-batch number 459000
Training mini-batch number 460000
Training mini-batch number 461000
Training mini-batch number 462000
Training mini-batch number 463000
Training mini-batch number 464000
Training mini-batch number 465000
Training mini-batch number 466000
Training mini-batch number 467000
Training mini-batch number 468000
Training mini-batch number 469000
Training mini-batch number 470000
Training mini-batch number 471000
Training mini-batch number 472000
Training mini-batch number 473000
Training mini-batch number 474000
Epoch 18: validation accuracy 99.20%
Training mini-batch number 475000
Training mini-batch number 476000
Training

Training mini-batch number 678000
Training mini-batch number 679000
Training mini-batch number 680000
Training mini-batch number 681000
Training mini-batch number 682000
Training mini-batch number 683000
Training mini-batch number 684000
Training mini-batch number 685000
Training mini-batch number 686000
Training mini-batch number 687000
Training mini-batch number 688000
Training mini-batch number 689000
Training mini-batch number 690000
Training mini-batch number 691000
Training mini-batch number 692000
Training mini-batch number 693000
Training mini-batch number 694000
Training mini-batch number 695000
Training mini-batch number 696000
Training mini-batch number 697000
Training mini-batch number 698000
Training mini-batch number 699000
Epoch 27: validation accuracy 99.23%
Training mini-batch number 700000
Training mini-batch number 701000
Training mini-batch number 702000
Training mini-batch number 703000
Training mini-batch number 704000
Training mini-batch number 705000
Training mi

Training mini-batch number 904000
Training mini-batch number 905000
Training mini-batch number 906000
Training mini-batch number 907000
Training mini-batch number 908000
Training mini-batch number 909000
Training mini-batch number 910000
Training mini-batch number 911000
Training mini-batch number 912000
Training mini-batch number 913000
Training mini-batch number 914000
Training mini-batch number 915000
Training mini-batch number 916000
Training mini-batch number 917000
Training mini-batch number 918000
Training mini-batch number 919000
Training mini-batch number 920000
Training mini-batch number 921000
Training mini-batch number 922000
Training mini-batch number 923000
Training mini-batch number 924000
Epoch 36: validation accuracy 99.30%
Training mini-batch number 925000
Training mini-batch number 926000
Training mini-batch number 927000
Training mini-batch number 928000
Training mini-batch number 929000
Training mini-batch number 930000
Training mini-batch number 931000
Training mi

Training mini-batch number 1132000
Training mini-batch number 1133000
Training mini-batch number 1134000
Training mini-batch number 1135000
Training mini-batch number 1136000
Training mini-batch number 1137000
Training mini-batch number 1138000
Training mini-batch number 1139000
Training mini-batch number 1140000
Training mini-batch number 1141000
Training mini-batch number 1142000
Training mini-batch number 1143000
Training mini-batch number 1144000
Training mini-batch number 1145000
Training mini-batch number 1146000
Training mini-batch number 1147000
Training mini-batch number 1148000
Training mini-batch number 1149000
Epoch 45: validation accuracy 99.42%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.47%
Training mini-batch number 1150000
Training mini-batch number 1151000
Training mini-batch number 1152000
Training mini-batch number 1153000
Training mini-batch number 1154000
Training mini-batch number 1155000
Training mini-batch number 1156000


KeyboardInterrupt: 

 经过观察到了epoch45以后交叉验证集没有明显变化，所以终止了代码运行
 
 添加两层全连接层的情况
 net = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
        FullyConnectedLayer(n_in=100, n_out=100, activation_fn=ReLU),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(expanded_training_data, 60, mini_batch_size, 0.03, 
            validation_data, test_data, lmbda=0.1)

添加两层全连接层，并对全连接层使用dropout=0.5。

In [9]:
net5 = Network([
        ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                      filter_shape=(20, 1, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                      filter_shape=(40, 20, 5, 5), 
                      poolsize=(2, 2), 
                      activation_fn=ReLU),
        FullyConnectedLayer(
            n_in=40*4*4, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        FullyConnectedLayer(
            n_in=1000, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        SoftmaxLayer(n_in=1000, n_out=10, p_dropout=0.5)], 
        mini_batch_size)
net5.SGD(expanded_training_data,40, mini_batch_size, 0.03, 
            validation_data, test_data)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-batch number 19000
Training mini-batch number 20000
Training mini-batch number 21000
Training mini-batch number 22000
Training mini-batch number 23000
Training mini-batch number 24000
Epoch 0: validation accuracy 98.67%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.68%
Training mini-batch number 25000
Training mini-batch number 2600

Training mini-batch number 216000
Training mini-batch number 217000
Training mini-batch number 218000
Training mini-batch number 219000
Training mini-batch number 220000
Training mini-batch number 221000
Training mini-batch number 222000
Training mini-batch number 223000
Training mini-batch number 224000
Epoch 8: validation accuracy 99.43%
This is the best validation accuracy to date.
The corresponding test accuracy is 99.55%
Training mini-batch number 225000
Training mini-batch number 226000
Training mini-batch number 227000
Training mini-batch number 228000
Training mini-batch number 229000
Training mini-batch number 230000
Training mini-batch number 231000
Training mini-batch number 232000
Training mini-batch number 233000
Training mini-batch number 234000
Training mini-batch number 235000
Training mini-batch number 236000
Training mini-batch number 237000
Training mini-batch number 238000
Training mini-batch number 239000
Training mini-batch number 240000
Training mini-batch number

Training mini-batch number 435000
Training mini-batch number 436000
Training mini-batch number 437000
Training mini-batch number 438000
Training mini-batch number 439000
Training mini-batch number 440000
Training mini-batch number 441000
Training mini-batch number 442000
Training mini-batch number 443000
Training mini-batch number 444000
Training mini-batch number 445000
Training mini-batch number 446000
Training mini-batch number 447000
Training mini-batch number 448000
Training mini-batch number 449000
Epoch 17: validation accuracy 99.46%
Training mini-batch number 450000
Training mini-batch number 451000
Training mini-batch number 452000
Training mini-batch number 453000
Training mini-batch number 454000
Training mini-batch number 455000
Training mini-batch number 456000
Training mini-batch number 457000
Training mini-batch number 458000
Training mini-batch number 459000
Training mini-batch number 460000
Training mini-batch number 461000
Training mini-batch number 462000
Training mi

KeyboardInterrupt: 